# Классификация изображений элементов одежды с помощью CNN

В этой части урока мы построим и обучим нейронную сеть классифицировать изображения элементов одежды, такие как платья, кроссовки, рубашки, футболки и т.п. В этом ноутбуке мы снова используем `tf.keras` - высокоуровневый API для построения и тренировки моделей в TensorFlow.

Всё впорядке, если какие-то моменты будут не очень понятны.

## Установка и импорт зависимостей

Нам понадобится **TensorFlow** и несколько вспомогательных библиотек.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

import logging
logging.getLogger('tensorflow').disabled = True

Загружаем датасет "Fashion MNIST", нормализуем данные. Значение каждого пикселя в изображении находится в интервале [0,255]. Для того, чтобы модель работала корректно эти значения необходимо нормализовать - привести к значениям в интервале [0,1].

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0


##Набор данных Fashion MNIST

В этом примере используется набор данных Fashion MNIST, который содержит 70 000 изображений элементов одежды в 10 категориях в градациях серого. Изображения содержат элементы одежды в низком разрешении (28х28 пикселей), как показано ниже:

![alt text](https://tensorflow.org/images/fashion-mnist-sprite.png)

Fashion MNIST используется как замена классическому набору данных MNIST - чаще всего используется как "Hello, World!" в машинном обучении и компьютерном зрении. Набор данных MNIST содержит изображения цифр написанных от руки (0, 1, 2 и тд) в таком же формате, каком представлены элементы одежды в нашем примере.

В нашем примере мы используем Fashion MNIST из-за разнообразия и потому, что эта задача более интересна с точки зрения реализации, чем решение типичной задачи на наборе данных MNIST. Оба наборы данных достаточно малы, поэтому используются для проверки корректной работоспособности алгоритма. Отличные наборы данных для старта изучения машинного обучения, тестирования и отладки кода.

Мы воспользуемся 60 000 изображениями для тренировки сети и 10 000 изображениями для проверки точности обучения и классификации изображений.

Выведем размеры массивов с данными:

In [3]:
print("Shape of Training Image Data: " + str(x_train.shape))
print("Shape of Training Class Data: " + str(y_train.shape))
print("Shape of Test Image Data: " + str(x_test.shape))
print("Shape of Test Class Data: " + str(y_test.shape))

Shape of Training Image Data: (60000, 28, 28)
Shape of Training Class Data: (60000,)
Shape of Test Image Data: (10000, 28, 28)
Shape of Test Class Data: (10000,)


Изображения представляют собой двумерные массивы 28х28, где значения в каждой ячейке могут быть в интервале `[0, 255]`. Метки классов - массив целых чисел, где каждое значение в интервале `[0, 9]`. Эти метки соответствуют выходному классу изображения следующим образом:

<table>
  <tr>
    <th>Метка</th>
    <th>Класс</th>
  </tr>
  <tr>
    <td>0</td>
    <td>Футболка / топ</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Шорты</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Свитер</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Платье</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Плащ</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Сандали</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Рубашка</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Кроссовок</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Сумка</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ботинок</td>
  </tr>
</table>

Каждое изображение относится к одной метке. Так как наименования классов не содержатся в исходном наборе данных, давайте сохраним их для дальнейшего использования, когда будем отрисовывать изображения:



In [4]:
class_names = ['Футболка / топ', "Шорты", "Свитер", "Платье",
              "Плащ", "Сандали", "Рубашка", "Кроссовок", "Сумка",
              "Ботинок"]

### Изучаем данные

Давайте отрисуем какое-нибудь одно изображение, чтобы взглянуть на него:

In [ ]:
# Берём первое изображение.
index = 0
plt.figure(figsize=(20,16))
plt.imshow(x_train[index], cmap=plt.cm.binary)
plt.xlabel(class_names[y_train[index]])
plt.colorbar()

ax = plt.gca()
ax.set_xticks(np.arange(-.5, 28, 1))
ax.set_yticks(np.arange(-.5, 28, 1))
ax.set_xticklabels(np.arange(0, 29, 1))
ax.set_yticklabels(np.arange(0, 29, 1))
ax.xaxis.tick_top()

plt.show()

Отобразим первые 25 изображений из тренировочного набора данных и под каждым изображением укажем к какому классу оно относится.

Убедитесь, что данные в корректном формате и мы готовы приступить к созданию и обучению сети.

In [ ]:
plt.figure(figsize=(15,15))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

### Строим модель

Построение нейронной сети требует настройки слоёв, а затем сборки модели с функциями оптимизации и потерь.

###Настраиваем слои

Базовым элементом при построении нейронной сети является *слой*. Слой извлекает представление из данных, которые поступили ему на вход.

Большую часть времени занимаясь глубоким обучением вы будете заниматься созданием связей между слоями. Большинство слоёв, например, такие как `tf.keras.layers.Dense` имеют набор параметров, которые могут быть "подогнаны" во время процесса обучения.

In [7]:
model = tf.keras.Sequential([
    # Ниже для слоя Conv2D используется 32 фильтра с ядрами 3х3 пикселя каждый. Параметр padding=’same’ означает,
    # что выходная карта признаков на каждом канале должна быть той же размерностью, что и исходное изображение,
    # т.е. 28х28 элементов. Фактически, это означает добавление значений на границах двумерных данных (обычно нулей),
    # чтобы центр ядра фильтра мог размещаться над граничными элементами, используется функция активации ReLu
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    # Следующий слой должен укрупнять масштаб полученных признаков. Для этого чаще всего используется операция MaxPooling.
    # Здесь pool_size (2, 2) - размер окна, в котором выбирается максимальное значение; strides – шаг сканирования по координатам плоскости
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    # Снова слой Conv2D, но уже используется 64 фильтра
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    # Снова слой MaxPooling2D
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

**ВЫходная часть сети состоит из трёх слоёв:**

* `tf.keras.layers.Flatten` - этот слой преобразует изображения размером 28х28 пикселей в 1D-массив размером 784 (28 * 28). На этом слое у нас нет никаких параметров для обучения, так как этот слой занимается только преобразованием входных данных.

* **скрытый слой** `tf.keras.layers.Dense` - плотносвязный слой из 128 нейронов. Каждый нейрон (узел) принимает на вход все 784 значения с предыдущего слоя, изменяет входные значения согласно внутренним весам и смещениям во время тренировки и возвращает единственное значение на следующий слой.

* **выходной слой** `ts.keras.layers.Dense` - `softmax`-слой состоит из 10 нейронов, каждый из которых представляет определённый класс элемента одежды. Как и в предыдущем слое, каждый нейрон принимает на вход значения всех 128 нейронов предыдущего слоя. Веса и смещения каждого нейрона на этом слое изменяются при обучении таким образом, чтобы результатирующее значение было в интервале `[0,1]` и представляло собой вероятность того, что изображение относится  к этому классу. Сумма всех выходных значений 10 нейронов равна 1.

###Компилируем модель

Перед тем как мы приступим к обучению модели, стоит ещё выполнить несколько настроек. Эти настройки производятся во время сборки модели при вызове метода `compile`:

* *функция потерь* - алгоритм измерения того, насколько далеко находится желаемое значение от спрогнозированного.
* *функция оптимизации* - агоритм "подгонки" внутренних параметров (весов и смещений) модели для минимизации функции потерь;
* *метрики* - используются для мониторинга процесса тренировки и тестирования. Пример ниже использует такую метрику как `точность`, процент изображений, которые были корректно классифицированы.

In [8]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

## Визуализируем модель

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=False)

## Обучаем модель

Модель учится сопоставлять входное изображение с меткой класса.
Параметр `epochs=10` ограничивает количество эпох обучения до 10 полных обучающих итераций по набору данных, что в итоге даёт нам тренировку на 10 * 60000 = 600 000 примерах.


In [ ]:
# Добавляем пустое цветовое измерение, так как сверточная сеть требует этого.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

history = model.fit(

      # Данные для обучения: изображения и их классы.
      x_train, y_train,

      # размер батча
      batch_size=64,

      # количество эпох обучения
      epochs=10,

      # Модель выделит часть обучающих данных (20%) и не будет на ней обучаться,
      # а будет оценивать ошибки модели на основе этих данных в конце каждой эпохи (так называемвя валидация).
      validation_split=0.2,

      verbose=1)

В процессе обучения модели значение функции потерь и метрика точности отображаются для каждой обучающей итерации.

### Проверяем точность

Проверим, какую точность выдаёт модель на тестовых данных. Воспользуемся всеми примерами, которые у нас есть в тестовом наборе данных для проверки точности.

In [ ]:
predicted_classes = model.predict(x_test)
classes=np.argmax(predicted_classes, axis=1)
print(classification_report(y_test, classes, target_names=class_names))

Как вы можете заметить, точность на тестовом наборе данных оказалась меньше точности на тренировочном наборе данных. Это вполне нормально, так как когда модель обнаруживает изображения, которые она ранее никогда не видела, вполне очевидно, что эффективность классификации снизится. Можете савмостоятельно почитать, что такое **precision, recall** и **f1-score**.

## Предсказываем и исследуем

Посмотрим примеры неправильно классифицированных тестовых данных:

In [ ]:
incorrect = np.nonzero(classes!=y_test)[0]

plt.figure(figsize=(15, 8))
for j, incorrect in enumerate(incorrect[0:8]):
    plt.subplot(2, 4, j+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[incorrect].reshape(28, 28), cmap="Reds")
    plt.title("Предсказано: {}".format(class_names[classes[incorrect]]))
    plt.xlabel("Правильно: {}".format(class_names[y_test[incorrect]]))

Воспользуемся обученой моделью, чтобы предсказать метку класса для единственного изображения:

In [ ]:
img = x_test[0]

print(img.shape)

Модели в `tf.keras` оптимизированы для предсказаний блоками (коллекциями). Поэтому, несмотря на то, что мы используем единственный элемент необходимо его добавить в список:

In [ ]:
img = np.array([img])

print(img.shape)

Теперь предскажем результат:

In [ ]:
predictions_single = model.predict(img)

print(predictions_single)

Метод `model.predict` возвращает список списков (массив массивов), каждый для изображения из блока входных данных. Получим единственный результат для нашего одного входного изображения:

In [ ]:
np.argmax(predictions_single[0])

Модель предсказала метку 9 (ботинок).

# Упражнения

Поэкспериментируйте с различными параметрами и посмотрите как будет меняться точность. В частности, попробуйте изменить следующее:

* установите параметр `epochs` равным 50;
* измените количество нейронов в скрытом слое (слой tf.keras.layers.Dense(128, activation=tf.nn.relu),), например, от низкого значения 32 до 512 и посмотрите, каким образом будет меняться точность прогноза модели;
* добавьте дополнительный слой между flatten-слоем (сглаживающим слоем) и конечным dense-слоем, проведите эксперименты с количеством нейронов на этом слое;
* не нормализуйте значения пикселей (не делите на 255) и посмотрите, что из этого получится,
* поэкспериментируйте с функцией оптимизации - попробуйте вместо adam использовать: adamax, adagrad, nadam...


# Выведем карты признаков из сверточного слоя

По сути, мы здесь копируем часть нашей обученной модели — слоя Conv2D. Затем мы подаем тестовое изображение и визуализируем выходные данные Conv2D — наши карты признаков.

In [ ]:
# Слой, который мы хотим скопировать из обученной сверточной сети.
layer_name = 'conv2d'

# Получаем список слоев из нашей модели
layer_dict = {layer.name : layer for layer in model.layers}

# Создаем копию существующей модели, содержащую только слой Conv2D.
modelslice = tf.keras.Model(inputs=model.inputs, outputs=layer_dict[layer_name].output)

# Выбераем изображение (от 0 до 59999) из обучающего набора.
image = x_train[0] # Берем первое

# Добавляем дополнительное измерение
image = np.expand_dims(image, axis=0)

# Подаем изображение на вход модели
feature_maps = modelslice.predict(image)

plt.figure(figsize=(15, 8))

# Мы предполагаем, что у нас есть 32 карты признаков.
for i in range(32):
    plt.subplot(4,8,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(feature_maps[0, :, :, i-1], cmap=plt.cm.binary)